# Analysis of NeurIPS 2023 Accepted Papers

In [ ]:
# !pip install wordcloud

In [ ]:
import json
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc_file('matplotlibrc')

from wordcloud import WordCloud

## Loading the Data

In [ ]:
with open("data/neurips2022data.json", "r") as f:
    neurips_2022_data = json.load(f)

with open("data/neurips2023data.json", "r") as f:
    neurips_2023_data = json.load(f)

with open("data/stopwords.txt", "r") as f:
    stopwords = f.read().split("\n")

In [ ]:
titles_2022 = [d.lower() for d in neurips_2022_data.keys()]
titles_2023 = [d.lower() for d in neurips_2023_data.keys()]

## Wordclouds

In [ ]:
def generate_wordcloud(titles, stopwords, filename):
    title_text = ' '.join(titles)
    title_text = ' '.join([word for word in title_text.split() if word not in stopwords])

    wordcloud = WordCloud(
        max_font_size=100, 
        min_font_size = 10, 
        max_words=100, 
        background_color="white", 
        width = 533*2, 
        height = 253*2, 
        mode="RGBA"
        ).generate(title_text)
    wordcloud.to_file(filename)

In [ ]:
for year, titles in zip(["2022", "2023"], [titles_2022, titles_2023]):
    generate_wordcloud(titles, stopwords, f"images/wordcloud_{year}.png");

### 2023 Title Wordcloud

![Neurips 2023 Title Wordcloud](images/wordcloud_2023.png)

## Abstract Lengths

In [ ]:
keys = list(neurips_2023_data.keys())

abstracts = [neurips_2023_data[key]["abstract"] for key in keys]
abstracts = [abstract.lower() for abstract in abstracts if abstract!=""]
abstract_lengths = [len(abstract.split()) for abstract in abstracts]

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(abstract_lengths, bins=100, density=True);
plt.xlabel("Abstract Length (words)");
plt.ylabel("Density");
plt.title("Abstract Lengths for NeurIPS 2023 Papers");
plt.savefig("images/abstract_histogram_2023.png");

In [ ]:
sorted_abstracts = sorted(zip(abstract_lengths, abstracts), key=lambda x: x[0], reverse=True)

Longest abstract:

In [ ]:
print(sorted_abstracts[0][1])
print("-"*100)

print(f"Num words {len(sorted_abstracts[0][1].split())}")

☝️ From "[Re] On the Reproducibility of \u201cFairCal: Fairness Calibration for Face Verification\u201d"

Shortest abstract:

In [ ]:
print(sorted_abstracts[-1][1])

print(f"Num words {len(sorted_abstracts[-1][1].split())}")

☝️ This is a spotlight poster: "Improved Convergence in High Probability of Clipped Gradient Methods with Heavy Tailed Noise"

## Number of Authors

In [ ]:
num_authors_2022 = [
    len(v) for v in neurips_2022_data.values()
]

num_authors_2023 = [
    len(neurips_2023_data[k]['authors']) for k in keys
]

In [ ]:
from collections import Counter

author_counts_2022 = Counter(num_authors_2022)
author_counts_2023 = Counter(num_authors_2023)

In [ ]:
all_keys = sorted(set(author_counts_2022.keys()).union(set(author_counts_2023.keys())))
values_2022 = [author_counts_2022.get(k, 0) for k in all_keys]
values_2023 = [author_counts_2023.get(k, 0) for k in all_keys]

# Set up the bar width and positions
bar_width = 0.35
r1 = range(len(all_keys))
r2 = [x + bar_width for x in r1]

# Create bar chart
plt.figure(figsize=(15, 6))
plt.bar(r1, values_2022, color='blue', width=bar_width, edgecolor='grey', label='2022')
plt.bar(r2, values_2023, color='red', width=bar_width, edgecolor='grey', label='2023')
# Add labels and title
plt.xlabel('Number of Authors')
plt.ylabel('Number of Papers')
plt.title('Comparison of Number of Authors per Paper for 2022 vs 2023')
plt.xticks([r + bar_width / 2 for r in r1], all_keys);

# Add a legend
plt.legend();
plt.savefig("images/num_authors_2022_2023.png")

Apparently there were some papers in 2023 with 1 author, when there were none in 2022.

#### Mean Number of Authors

In [ ]:
mean_authors_2022 = np.mean(num_authors_2022)
mean_authors_2023 = np.mean(num_authors_2023)

print(f"Mean number of authors per paper in 2022: {mean_authors_2022}")
print(f"Mean number of authors per paper in 2023: {mean_authors_2023}")

Despite the single author papers, the mean number of authors in 2023 is still higher than in 2022.

#### Paper with the Most Authors

In [ ]:
neurips_2023_data[keys[int(np.argmax(num_authors_2023))]]["title"]

### Most Prolific Authors

*Note*: This assumes that the author names are consistent across papers, and that the author names are unique, which is not always the case!

In [ ]:
unique_authors_2022 = set()
for v in neurips_2022_data.values():
    unique_authors_2022.update(v)

unique_authors_2023 = set()
for v in neurips_2023_data.values():
    unique_authors_2023.update(v["authors"])

print(f"Number of unique authors in 2022: {len(unique_authors_2022)}")
print(f"Number of unique authors in 2023: {len(unique_authors_2023)}")

authors_counts = {
    author_name: 0 for author_name in unique_authors_2023
}

for v in neurips_2023_data.values():
    for author in v["authors"]:
        authors_counts[author] += 1

authors_counts = sorted(authors_counts.items(), key=lambda x: x[1], reverse=True)

In [ ]:
print(f"Most prolific authors in 2023:")
print("-----------------------------")
for author, count in authors_counts[:10]:
    print(f"{author}: {count} papers")

## Title Length Comparison between 2022 and 2023

In [ ]:
title_lengths_2022 = [len(title.split()) for title in titles_2022]
title_lengths2023 = [len(neurips_2023_data[key]["title"].split()) for key in keys]

plt.figure(figsize=(10, 6))
plt.hist(title_lengths_2022, bins=100, density=True, label="2022");
plt.hist(title_lengths2023, bins=100, density=True, label="2023");
plt.xlabel("Title Length (words)");
plt.ylabel("Density");
plt.legend();
plt.savefig("images/title_length_histogram_2022_2023.png");

In [ ]:
mean_title_length_2022 = np.mean(title_lengths_2022)
mean_title_length_2023 = np.mean(title_lengths2023)

print(f"Mean title length in 2022: {mean_title_length_2022}")
print(f"Mean title length in 2023: {mean_title_length_2023}")

Titles are trending longer in 2023!

### Acronyms

In [ ]:
def _introduces_acronym(title):
    return ":" in title and len(title.split(":")[0].split("(")[0].split()) == 1

In [ ]:
num_acronyms_2022 = sum([_introduces_acronym(title) for title in titles_2022])
num_acronyms_2023 = sum([_introduces_acronym(neurips_2023_data[key]["title"]) for key in keys])

percentage_acronyms_2022 = num_acronyms_2022 / len(titles_2022)
percentage_acronyms_2023 = num_acronyms_2023 / len(keys)

print(f"Percentage of titles that introduce acronyms in 2022: {percentage_acronyms_2022}")
print(f"Percentage of titles that introduce acronyms in 2023: {percentage_acronyms_2023}")

Even the acronyms per paper are trending up!

### Titles with LaTeX

In [ ]:
num_titles_with_latex_2022 = sum(["$" in title for title in titles_2022])
num_titles_with_latex_2023 = sum(["$" in neurips_2023_data[key]["title"] for key in keys])

percentage_titles_with_latex_2022 = num_titles_with_latex_2022 / len(titles_2022)
percentage_titles_with_latex_2023 = num_titles_with_latex_2023 / len(keys)

print(f"Percentage of titles that contain LaTeX in 2022: {percentage_titles_with_latex_2022}")
print(f"Percentage of titles that contain LaTeX in 2023: {percentage_titles_with_latex_2023}")

Whereas no titles had LaTeX in 2022, about 1% of titles in 2023 have LaTeX.

## High-Level Analysis of Abstracts

In [ ]:
abstracts = [neurips_2023_data[key]["abstract"].lower() for key in keys]

In [ ]:
def _count_occurrences_of_words(abstracts, words):
    '''
    Counts the number of abstracts that contain at least one 
    of the words in the list of words.
    '''
    i = 0
    for abstract in abstracts:
        for word in words:
            if word in abstract:
                i += 1
                break
    return i

def _print_occurrences_of_words(abstracts, words):
    '''
    Prints the number of abstracts that contain at least one 
    of the words in the list of words.
    '''
    num_occurrences_2023 = _count_occurrences_of_words(abstracts, words)
    print(f"# occurrences of {words[0]} words: {num_occurrences_2023}")

In [ ]:
BROAD_STROKES_KEYWORDS = (
    ["sampling"],
    ["differentiable", "differentiation"],
    ["model"],
    ["data"],
    ["benchmark"],
    ["representation"],
    ["robust", "robustness", "robustly"],
    ["learning", "learn"],
    ["foundation"],
)

MODALITY_KEYWORDS = (
    ["vision", "visual"],
    ["language"],
    ["audio"],
    ["text"],
    ["video"],
    ["speech"],
    ["time series", "time-series", "temporal"],
    ["multi-modal", "multimodal"],
    ["cross-modal", "crossmodal"],
)

TASK_KEYWORDS = (
    ["classification", "classify"],
    ["regression", "regress"],
    ["generation", "generate"],
    ["translation", "translate"],
    ["segmentation", "segment"],
    ["detection", "detect"],
    ["localization", "localize"],
    ["reconstruction", "reconstruct"],
    ["representation", "represent"],
    ["embedding", "embed"],
    ["prediction", "predict"],
    ["synthesis", "synthesize"],
    ["imitation", "imitate"],
    ["plan"],
    ["control"],
    ["search"],
    ["optimization", "optimize"],
    ["adversarial", "adversary", "adversaries"],
    ["generative", "generation"],
    ["privacy", "private"],
    ["distill"],
    ["federated"],
    ["adapt"],
    ["transfer"],
    ["meta"],
    ["few-shot", "few shot"],
    ["zero-shot", "zero shot"],
    ["fine-tuning", "finetuning"],
    ["reinforcement learning", " rl ", "(rl)"],
    ["multi-task", "multitask"],
)

THEORY_KEYWORDS = (
    ["bayes"],
    ["monte-carlo", "monte carlo"],
    ["gauss"],
    ["efficient", "efficiency"],
    ["supervised", "supervision"],
)

MODEL_KEYWORDS = (
    ["clip", "contrastive language-image pre-training"],
    ["nerf", "neural radiance fields"],
    ["transformer"],
    ["llm", "large language model"],
    ["gpt"],
    ["gnn", "graph neural network"],
    ["agent"]
)

LANGUAGE_KEYWORDS = [
    ["open"],
    ["vocabulary"],
    ["prompt"],
    ["grounding", "grounded"],
]

DIMENSION_KEYWORDS = [
    ["2d"],
    ["3d"],
    ["4d"],
    ["6d"],
]

MULTIMODAL_KEYWORDS = [
    ["vision-language", "vision language"],
    ["text-to-image", "text to image", "t2i"],
    ["image-to-text", "image to text", "i2t"],
    ["text-to-video", "text to video", "t2v"],
    ["captioning"],
    ["visual question answering", "vqa"],
]
 

### Broad Topics

In [ ]:
print("--------BROAD STROKES--------")
for words in BROAD_STROKES_KEYWORDS:
    _print_occurrences_of_words(abstracts, words)

### Modalities

In [ ]:
print("--------MODALITY--------")
for words in MODALITY_KEYWORDS:
    _print_occurrences_of_words(abstracts, words)

### Tasks

In [ ]:
print("--------TASK--------")
for words in TASK_KEYWORDS:
    _print_occurrences_of_words(abstracts, words)
    

### Theory

In [ ]:
print("--------THEORY--------")
for words in THEORY_KEYWORDS:
    _print_occurrences_of_words(abstracts, words)

### Models

In [ ]:
print("--------MODEL--------")
for words in MODEL_KEYWORDS:
    _print_occurrences_of_words(abstracts, words)   

### Language

In [ ]:
print("--------LANGUAGE--------")
for words in LANGUAGE_KEYWORDS:
    _print_occurrences_of_words(abstracts, words)

### Dimensions

In [ ]:
print("--------DIMENSION--------")
for words in DIMENSION_KEYWORDS:
    _print_occurrences_of_words(abstracts, words)

### Multimodal

In [ ]:
print("--------MULTIMODAL--------")
for words in MULTIMODAL_KEYWORDS:
    _print_occurrences_of_words(abstracts, words)